In [174]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [175]:
class JumpVasicek:
    """Vasicek Model with jumps"""
    def __init__(self, model_params: dict):
        self.model_params = model_params
    def increment(self, rj: float, dt: float, nj=None, Pj=None, Jj=None):
        """
        Calculates next interest rate step
        """
        if nj is None or Pj is None or Jj is None:
            nj = np.random.normal()
            Pj = np.random.poisson(model_params["h"]*dt)
            Jj = np.random.normal(model_params["mu"], model_params["gamma"])
        time_step = self.model_params["kappa"]*(self.model_params["mu_r"] - rj)*dt
        stoch_step = self.model_params["sigma"]*np.sqrt(dt)*nj
        jump_step = Jj*Pj
        return rj + time_step + stoch_step + jump_step, nj, Pj, Jj

In [176]:
class Vasicek:
    """Canonical Vasicek Model"""
    def __init__(self, model_params: dict):
        self.model_params = model_params
    def increment(self, rj, dt, nj=None, Pj=None, Jj=None):
        if nj is None:
            nj = np.random.normal()
            Pj = np.random.poisson(model_params["h"]*dt)
            Jj = np.random.normal(model_params["mu"], model_params["gamma"])
        time_step = self.model_params["kappa"]*(self.model_params["mu_r"] - rj)*dt
        stoch_step = self.model_params["sigma"]*np.sqrt(dt)*nj
        return rj + time_step + stoch_step, nj, Pj, Jj
    def exact(self, r0, T):
        """
        Returns exact price rate for maturity T
        """
        K = self.model_params["kappa"]
        u_hat = self.model_params["mu_r"]
        B = (1 - np.exp(-K*(T)))/K
        A = (B - (T))*(u_hat - self.model_params["sigma"]**2/(2*K*K)) - (self.model_params["sigma"]**2)*(B**2)/(4*K)
        return np.exp(A - B*r0)

In [177]:
class MonteCarlo:
    def __init__(self, model):
        self.model = model
    def _generate_path(self, r0, T: float, m: int, n=None, P=None, J=None):
        """
        r0, float: i-rate today
        T, float: Maturity date (in years)
        m, int: time steps per year,
        n, 1x(T*m) np.array, array of draws,
        """
        dt = 1/m
        num_steps = int(np.ceil(T/dt))
        r = np.empty(num_steps + 1)
        none = False
        if n is None:
            n = np.empty(num_steps)
            P = np.empty(num_steps)
            J = np.empty(num_steps)
            none = True
        r[0] = r0
        for j in range(num_steps):
            r[j+1], n[j], P[j], J[j] = self.model.increment(rj=r[j], dt=dt, 
                                                nj=-n[j] if not none else None,
                                                Pj=P[j] if not none else None,
                                                Jj=-J[j] if not none else None,
                                                )
        return r, n, P, J
    def _evaluate_price(self, r: np.array, m: int):
        """
        r, 1xk np.array: i-rate path,
        m, int: time steps per year
        """
        dt = 1/m
        return np.exp(-np.sum(np.multiply(r, dt)))
    def _simulate_paths(self, m: int, r0: float, n: int, T: float):
        """
        m, int: time steps per year
        r0, float: i-rate today
        n, int: number of simulations
        T, float: Maturity date (in years)
        """
        prices = np.empty(n)
        for i in range(n):
            r, _, __, ___ = self._generate_path(r0, T, m)
            prices[i] = self._evaluate_price(r, m)
        return np.mean(prices), np.std(prices)
    def _simulate_paths_anti(self, m: int, r0: float, n: int, T: float):
        """
        m, int: time steps per year
        r0, float: i-rate today
        n, int: number of simulations
        T, float: Maturity date (in years)
        """
        prices = np.empty(n)
        for i in range(n):
            r1, n, P, J = self._generate_path(r0, T, m)
            r2, _, __, ___ = self._generate_path(r0, T, m, n, P, J)
            prices[i] = 0.5*(self._evaluate_price(r1, m) + self._evaluate_price(r2, m))
        return np.mean(prices), np.std(prices)

In [178]:
model_params = {
    "kappa": 0.5,
    "mu_r": 0.06,
    "sigma": 0.02,
    "mu": 0,
    "gamma": 0.01,
    "h": 10
}

In [179]:
vasicek = Vasicek(model_params)
mc_vasicek = MonteCarlo(vasicek)

In [180]:
mc_vasicek._simulate_paths(m=365, r0=0.04, n=1000, T=3)

(0.8606140749100691, 0.0301751539292666)

In [181]:
mc_vasicek._simulate_paths_anti(m=365, r0=0.04, n=1000, T=3)

(0.8620573169970779, 0.00077763179139923)

In [182]:
vasicek.exact(r0=0.04, T=3)

0.8622146550422263

In [183]:
jump_vasicek = JumpVasicek(model_params)
mc_jump_vasicek = MonteCarlo(jump_vasicek)

In [184]:
mc_jump_vasicek._simulate_paths(m=365, r0=0.04, n=1000, T=3)

(0.8643673368088876, 0.06087874500953521)

In [185]:
mc_jump_vasicek._simulate_paths_anti(m=365, r0=0.04, n=1000, T=3)

(0.8635177916514604, 0.002944436636085731)